In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# Implementing Single RNN layer

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b # h_(t-1) * W_h + X_t * W_t = h_t
        h_next = np.tahh(t) # tanh for activation

        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

In [11]:
# "Actual" RNN
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]

        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 합산된 기울기
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

In [15]:
# Simple RNNLM
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *

class SimpleRNNLM:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # Initializing weights
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # Layers
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]

        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # All weights and grads in list
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)

        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        
        return dout
    
    def reset_state(self):
        self.rnn_layer.reset_state()

In [16]:
# Training RNNLM
import sys
sys.path.append('..')
from common.optimizer import SGD
from dataset import ptb

# hyperparameters
batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5
lr = 0.1
max_epoch = 100

# reading training data (only 1000)
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1] # current word sequence
ts = corpus[1:] # next word sequence
data_size = len(xs)
print('말뭉치 크기: %d, 어휘 수: %d' % (corpus_size, vocab_size))

# variable for training
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# Creating model
model = SimpleRNNLM(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # mini-batch
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1

        # calculate grad and update parameters
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1

    # evaluate perplexity per epoch
    ppl = np.exp(total_loss / loss_count)
    print('| EPOCH %d | PERPLEXITY %.2f' % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

말뭉치 크기: 1000, 어휘 수: 418
| EPOCH 1 | PERPLEXITY 389.79
| EPOCH 2 | PERPLEXITY 251.15
| EPOCH 3 | PERPLEXITY 218.73
| EPOCH 4 | PERPLEXITY 212.30
| EPOCH 5 | PERPLEXITY 204.20
| EPOCH 6 | PERPLEXITY 201.52
| EPOCH 7 | PERPLEXITY 197.91
| EPOCH 8 | PERPLEXITY 196.09
| EPOCH 9 | PERPLEXITY 191.86
| EPOCH 10 | PERPLEXITY 193.13
| EPOCH 11 | PERPLEXITY 188.57
| EPOCH 12 | PERPLEXITY 192.47
| EPOCH 13 | PERPLEXITY 190.35
| EPOCH 14 | PERPLEXITY 191.32
| EPOCH 15 | PERPLEXITY 190.33
| EPOCH 16 | PERPLEXITY 187.13
| EPOCH 17 | PERPLEXITY 184.79
| EPOCH 18 | PERPLEXITY 181.71
| EPOCH 19 | PERPLEXITY 183.62
| EPOCH 20 | PERPLEXITY 184.35
| EPOCH 21 | PERPLEXITY 182.73
| EPOCH 22 | PERPLEXITY 178.70
| EPOCH 23 | PERPLEXITY 175.66
| EPOCH 24 | PERPLEXITY 177.10
| EPOCH 25 | PERPLEXITY 174.61
| EPOCH 26 | PERPLEXITY 175.53
| EPOCH 27 | PERPLEXITY 170.15
| EPOCH 28 | PERPLEXITY 169.20
| EPOCH 29 | PERPLEXITY 168.69
| EPOCH 30 | PERPLEXITY 161.18
| EPOCH 31 | PERPLEXITY 163.37
| EPOCH 32 | PERPLEXITY 